In [1]:
import requests
import json
import random
import time
import pickle


model = "dRIT"

template = {
  "Answer": "",
  "Score Lexical Semantics": "",
  "Score Predicate-Argument Structure": "",
  "Score Logic": "",
  "Score Knowledge & Common Sense": ""
}


# TRAIN de Scitail

In [4]:
import pandas as pd

In [5]:
df =pd.read_pickle("predictions/RL_Scitail_TEST.pickle")
df

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases
0,2,0.625000,0.0,0.25,0.476290,0.111644,4.182847,0.643081,0.875000,3,0,0.542593,0,0,0.375000,neutral
1,2,0.666667,0.0,0.20,0.522233,0.046292,4.202914,0.570018,0.636364,2,0,0.598485,0,0,0.333333,neutral
2,2,0.600000,0.0,0.40,0.680414,0.189101,4.482404,0.927331,0.714286,0,1,0.500000,0,0,0.400000,neutral
3,2,0.800000,0.0,0.00,0.577350,0.053653,4.090163,0.543628,0.428571,0,0,0.722222,0,0,0.200000,neutral
4,2,0.750000,0.0,0.25,0.589256,0.094628,4.127185,0.561319,0.571429,1,0,0.565476,0,0,0.250000,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,2,0.571429,0.0,0.20,0.379473,0.009841,4.248794,0.693409,0.800000,3,0,0.461905,0,0,0.428571,neutral
2122,2,0.700000,0.0,0.25,0.600000,0.074261,4.268414,0.657199,0.666667,1,0,0.566667,0,0,0.300000,neutral
2123,2,0.750000,0.0,0.20,0.516398,0.074658,4.147063,0.605579,0.555556,1,0,0.603367,0,0,0.250000,neutral
2124,2,0.250000,0.0,0.00,0.566947,0.155856,4.457893,0.802590,1.000000,1,0,0.297619,0,0,0.750000,neutral


In [15]:
for i in range(3):
    if i==0:
        dev = pd.read_csv('../OPENAI/data/Scitail/TEST_'+str(i+1)+'.csv')
    else:
        try:
            temp = pd.read_csv('../OPENAI/data/Scitail/TEST_'+str(i+1)+'.csv')
            dev=pd.concat([dev,temp])
        except:
            print(i)

In [16]:
original =dev.copy().reset_index(drop=True)
original

,sentence1,sentence2,gold_label
0,Based on the list provided of the uses of subs...,"If a substance has a ph value greater than 7,t...",neutral
1,If one or two base pairs are change...,Invertebrates (and higher animals) can also be...,neutral
2,"At high temperatures, the solid dye converts i...",Gases and liquids become solids at low tempera...,neutral
3,Chapter 11 Gas and Kinetic Theory .,The behavior of ideal gases is explained by ki...,neutral
4,Both the continental crust and the oceanic cru...,Gabbro is a dark dense rock that can be found ...,neutral
...,...,...,...
2121,"In addition, the small pieces of prairie mean ...",A fewer predators is most likely to cause the ...,neutral
2122,D structures will help scientists determine be...,Studies of earthquake waves have helped scient...,neutral
2123,Of particular relevance is the role of terrest...,A primary role of decomposers in an ecosystem ...,neutral
2124,A new public gazebo at the water's edge is one...,One example of matter is water.,neutral


In [18]:
original["gold_label"].unique()

array(['neutral', 'entails'], dtype=object)

In [19]:
df["sentence_A"]=original["sentence1"]
df["sentence_B"]=original["sentence2"]

In [20]:
entailment=df[df["clases"]=="entails"]
neutral=df[df["clases"]!="entails"]
neutral

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
0,2,0.625000,0.0,0.25,0.476290,0.111644,4.182847,0.643081,0.875000,3,0,0.542593,0,0,0.375000,neutral,Based on the list provided of the uses of subs...,"If a substance has a ph value greater than 7,t..."
1,2,0.666667,0.0,0.20,0.522233,0.046292,4.202914,0.570018,0.636364,2,0,0.598485,0,0,0.333333,neutral,If one or two base pairs are change...,Invertebrates (and higher animals) can also be...
2,2,0.600000,0.0,0.40,0.680414,0.189101,4.482404,0.927331,0.714286,0,1,0.500000,0,0,0.400000,neutral,"At high temperatures, the solid dye converts i...",Gases and liquids become solids at low tempera...
3,2,0.800000,0.0,0.00,0.577350,0.053653,4.090163,0.543628,0.428571,0,0,0.722222,0,0,0.200000,neutral,Chapter 11 Gas and Kinetic Theory .,The behavior of ideal gases is explained by ki...
4,2,0.750000,0.0,0.25,0.589256,0.094628,4.127185,0.561319,0.571429,1,0,0.565476,0,0,0.250000,neutral,Both the continental crust and the oceanic cru...,Gabbro is a dark dense rock that can be found ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,2,0.571429,0.0,0.20,0.379473,0.009841,4.248794,0.693409,0.800000,3,0,0.461905,0,0,0.428571,neutral,"In addition, the small pieces of prairie mean ...",A fewer predators is most likely to cause the ...
2122,2,0.700000,0.0,0.25,0.600000,0.074261,4.268414,0.657199,0.666667,1,0,0.566667,0,0,0.300000,neutral,D structures will help scientists determine be...,Studies of earthquake waves have helped scient...
2123,2,0.750000,0.0,0.20,0.516398,0.074658,4.147063,0.605579,0.555556,1,0,0.603367,0,0,0.250000,neutral,Of particular relevance is the role of terrest...,A primary role of decomposers in an ecosystem ...
2124,2,0.250000,0.0,0.00,0.566947,0.155856,4.457893,0.802590,1.000000,1,0,0.297619,0,0,0.750000,neutral,A new public gazebo at the water's edge is one...,One example of matter is water.


In [21]:
df_e=entailment.sample(n=300)
df_n=neutral.sample(n=300)
#df_c=contradiction.sample(n=200)
df_t=pd.concat([df_e,df_n])
df_t

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
1118,2,0.333333,0.0,0.666667,0.755929,0.232549,4.445956,0.770646,1.000000,1,0,0.000000,0,0,0.666667,entails,Introduction An invertebrate is an animal with...,Backbone is another name for the vertebral col...
2066,2,0.333333,0.0,0.000000,0.554700,0.168914,4.650058,1.000000,1.000000,0,0,0.493590,0,0,0.666667,entails,"Composed of DNA, genes are the basic unit of i...",Genes are composed of dna.
501,1,0.000000,0.0,0.333333,0.632456,0.223126,4.411628,0.966823,1.000000,1,0,0.683333,0,0,1.000000,entails,"Luckily, hydrogen is the most overwhelming abu...",The element hydrogen is the most abundant in t...
1429,2,0.500000,0.0,0.250000,0.408248,0.105719,4.205463,0.704509,0.833333,2,0,0.578704,0,0,0.500000,entails,The equator is relatively hot because sunlight...,The equator on earth receives the most direct ...
129,2,0.500000,0.0,0.000000,0.296174,0.164021,4.317645,0.978230,1.000000,1,0,0.479310,0,0,0.500000,entails,"The team discovered that an immune cell, calle...",Lyme disease is caused by bacteria.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1862,2,0.714286,0.0,0.600000,0.547723,0.061328,4.124651,0.513348,0.500000,1,0,0.475000,0,0,0.285714,neutral,Increased fragmentation of habitats and urbani...,A fewer predators is most likely to cause the ...
1419,2,0.285714,0.0,0.200000,0.424264,0.128466,4.358496,0.740927,0.875000,2,0,0.459259,0,0,0.714286,neutral,When the North Pole is tilted furthest from th...,The number of daylight hours each day would be...
579,2,1.000000,0.0,0.333333,0.567775,0.130620,4.208711,0.641779,0.571429,0,0,0.569872,0,0,0.000000,neutral,He started studying kinetic gas theory and mov...,The behavior of ideal gases is explained by ki...
721,2,0.846154,0.0,0.125000,0.444444,0.098884,3.536910,0.447358,0.714286,6,0,0.482143,0,0,0.153846,neutral,This property is critical to high temperature ...,You call materials able to conduct electricity...


In [22]:
lista_respuestasOllama=[]

In [23]:
df.columns

Index(['relation', 'no_matcheadas', 'contradiction', 'jaccard', 'simBoW',
       'sumas', 'mutinf_t', 'max_info_t', 'entail', 'list_comp', 'list_incomp',
       'Jaro-Winkler_rit', 'negT', 'negH', 'overlap_ent', 'clases',
       'sentence_A', 'sentence_B'],
      dtype='object')

In [24]:
faltantes=[]

In [25]:
df_t=df_t.reset_index(drop=True)
df_t

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
0,2,0.333333,0.0,0.666667,0.755929,0.232549,4.445956,0.770646,1.000000,1,0,0.000000,0,0,0.666667,entails,Introduction An invertebrate is an animal with...,Backbone is another name for the vertebral col...
1,2,0.333333,0.0,0.000000,0.554700,0.168914,4.650058,1.000000,1.000000,0,0,0.493590,0,0,0.666667,entails,"Composed of DNA, genes are the basic unit of i...",Genes are composed of dna.
2,1,0.000000,0.0,0.333333,0.632456,0.223126,4.411628,0.966823,1.000000,1,0,0.683333,0,0,1.000000,entails,"Luckily, hydrogen is the most overwhelming abu...",The element hydrogen is the most abundant in t...
3,2,0.500000,0.0,0.250000,0.408248,0.105719,4.205463,0.704509,0.833333,2,0,0.578704,0,0,0.500000,entails,The equator is relatively hot because sunlight...,The equator on earth receives the most direct ...
4,2,0.500000,0.0,0.000000,0.296174,0.164021,4.317645,0.978230,1.000000,1,0,0.479310,0,0,0.500000,entails,"The team discovered that an immune cell, calle...",Lyme disease is caused by bacteria.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2,0.714286,0.0,0.600000,0.547723,0.061328,4.124651,0.513348,0.500000,1,0,0.475000,0,0,0.285714,neutral,Increased fragmentation of habitats and urbani...,A fewer predators is most likely to cause the ...
596,2,0.285714,0.0,0.200000,0.424264,0.128466,4.358496,0.740927,0.875000,2,0,0.459259,0,0,0.714286,neutral,When the North Pole is tilted furthest from th...,The number of daylight hours each day would be...
597,2,1.000000,0.0,0.333333,0.567775,0.130620,4.208711,0.641779,0.571429,0,0,0.569872,0,0,0.000000,neutral,He started studying kinetic gas theory and mov...,The behavior of ideal gases is explained by ki...
598,2,0.846154,0.0,0.125000,0.444444,0.098884,3.536910,0.447358,0.714286,6,0,0.482143,0,0,0.153846,neutral,This property is critical to high temperature ...,You call materials able to conduct electricity...


In [26]:
df_t.to_pickle("predictions/RL_Scitail_TEST_600.pickle")

# RIT base

In [27]:
for i in range(1):
    for index,strings in df_t[i*600:(i+1)*600].iterrows():
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=15)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("resultados/rit_base_Scitail_TEST_600_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*600,(i+1)*600)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [28]:
print(len(lista_respuestasOllama))

600
